In [45]:
import numpy as np
import os
import pandas as pd
from HySpec_Image_Processing import HDR_Image
from fancy_spec_plot import fancy_spec_plot
import matplotlib.pyplot as plt
from spec_average import spec_avg
from scipy import interpolate as interp
import numpy.random as r
from scipy import signal
import spectral as sp
import tifffile as tf
from fancy_spec_plot import plot_numpy_images
from copy import copy


def plot_numpy_images(*args:np.ndarray,**kwargs:np.ndarray):
    defaultKwargs = {"titles":[],"figsize":(8,16)}
    kwargs = {**defaultKwargs,**kwargs}

    fig = plt.figure(figsize=kwargs.get("figsize"))
    col_num = len(args)
    for num,image in enumerate(args):
        ax = fig.add_subplot(1,col_num,num+1)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(kwargs.get('titles')[num])
        ax.imshow(image)

mu = np.array([0,0])
cov = np.array(([1,0.5],[0.5,1]))
multi=np.random.multivariate_normal(mu,cov,10000)
x,y = multi.T[0],multi.T[1]
dist = np.histogram2d(x,y,bins=5)


def destripe(image,box_size,**kwargs):
    defaultKwargs = {"plotImg":False}
    kwargs = {**defaultKwargs,**kwargs}

    destripedImage = np.zeros(image.shape)
    n=0
    for row in range(image.shape[0]):
        xCoords = np.arange(image.shape[1])
        yAvg,std,xAvg = spec_avg(image[row,:,0],xCoords,box_size)
        f = interp.CubicSpline(xAvg,yAvg)
        xtest = np.linspace(0,image.shape[1],304)
        destripedImage[row,:,0] = f(xtest)
        if kwargs.get('plotImg') == True:
            if n<3:
                fig,ax = plt.subplots(1,1)
                ax.plot(xCoords,image[row,:,0])
                ax.plot(xAvg,yAvg)
                ax.plot(xtest,f(xtest))
            else:
                pass
            fig,ax = plt.subplots(1,1)
            ax.imshow(destripedImage[:,:,0])

        n+=1

    return destripedImage


   

hdr = sp.envi.open(r'D:\Data/20230209T095534013597/extracted_files/hdr_files/m3g20090417t193320_v01_rfl/m3g20090417t193320_v01_rfl.hdr')
bandCenters = hdr.bands.centers
bandCenters = np.array(bandCenters)

allowedIndices = np.where((bandCenters>900)&(bandCenters<2600))[0]
allowedWvl = bandCenters[allowedIndices]

image = hdr.read_bands(allowedIndices)

im1,im2,im3,im4 = destripe(image,3),destripe(image,5),destripe(image,7),destripe(image,10)

plot_numpy_images(im1,im2,im3,im4,titles=['3','5','7','10'])

ImportError: cannot import name 'plot_numpy_images' from 'fancy_spec_plot' (c:\Users\zacha\OneDrive\Desktop\M3_Pro\fancy_spec_plot.py)

In [48]:
from fancy_spec_plot import 